In [ ]:
'''
Moshe's idea:
1. Start with a large Boolean formula f (say, industrial SAT benchmark).
2. Generate many samples satisfying f or \neg f.
(Using, for example, [https://www.nature.com/articles/s42256-018-0002-3](https://www.nature.com/articles/s42256-018-0002-3))
3. Train a DNN on this labeled sample.
4. Check whether it is a good apprcximation of f.
----------
To use this notebook, please install pysat: https://pysathq.github.io/installation.html
Perhaps the simplest way is with the following command: pip install python-sat[pblib,aiger]
(pysat might not work on Windows)
'''


# This is just a warm-up cell, to see how pysat works

# some examples of pysat usage at: https://github.com/pysathq/pysat/blob/master/examples/usage.py
from pysat.solvers import Glucose3
g = Glucose3()
g.add_clause([-1, 2])
g.add_clause([-2, 3])


for m in g.enum_models():  # implemented as a generator
    print(m)
g.delete()

In [ ]:
# to use this cell, extract bw_large.d.cnf from instances/blocksworld.tar.gz
# blocksworld is from satlib at https://www.cs.ubc.ca/~hoos/SATLIB/Benchmarks/SAT/PLANNING/BlocksWorld/blocksworld.tar.gz
# satlib website: https://www.cs.ubc.ca/~hoos/SATLIB/benchm.html

import random
import pandas as pd
from pysat.solvers import Glucose3
from pysat.formula import CNF
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, precision_score


def generate_dataset(cnf_file):
    '''
    Receives a boolean formula in CNF format and does step 2 of the methodology:
    2. Generate many samples satisfying f or \neg f.
    
    Particularly, it enumerates all satisfying samples and the same number of unsat samples.
    Unsat samples are generated by flipping one bit of a sat sample.
    '''

    formula = CNF(from_file=cnf_file)

    num_vars = formula.nv
    sat_list = []
    unsat_list = []

    with Glucose3(bootstrap_with=formula) as solver:

        # for each positive (sat) instance, flip a literal to generate a negative (unsat) instance

        # transforming each sat instance into tuple eases the generation of negative instances
        sat_list = [m for m in solver.enum_models()] #enum_models returns a generator and not a list
        
        #print(sat_list[0])
        sat_set = set([tuple(s) for s in sat_list]) # much quicker to verify an existing instance

        for assignment in sat_list:
            for i, literal in enumerate(assignment):
                tentative_unsat = list(assignment)
                tentative_unsat[i] = -tentative_unsat[i] #negating one literal
                if tuple(tentative_unsat) not in sat_set:
                    unsat_list.append(tentative_unsat)
                    break # goes on to next assignment
                #print(f'negated {i}-th')

        #for num, m in enumerate(solver.enum_models()):  
        #print(num)
    #print(len(sat_list))
    #print(len(unsat_list))
    
    # appends 1 to sat samples, 0 to unsat samples
    for sat in sat_list:
        sat.append(1)
    for unsat in unsat_list:
        unsat.append(0)
    
    # concats and shuffles the two lists
    all_data = sat_list + unsat_list
    #random.seed(2) # uncomment to debug (otherwise each shuffle will give a different array)
    random.shuffle(all_data)
    
    # columns = [x1, x2, ..., xn, f]
    data_colnames = [f'x{i}' for i in range(1, len(all_data[0]))]
    df = pd.DataFrame(all_data, columns = data_colnames + ['f'])
    
    # replaces negatives by 0 and positives by 1
    df.mask(df < 0, 0, inplace=True)
    df.mask(df > 0, 1, inplace=True)
    
    # breaks into input features & label
    data_x = df.drop('f', axis=1)
    data_y = df['f']

    return data_x, data_y

def run_model(model, data_x, data_y, splitter=StratifiedKFold(n_splits=5)):
    '''
    Runs a machine learning model in the dataset
    '''
    
    # prints the header
    print('#instances\tprec\tacc')

    # trains the model for each split (fold)
    for train_index, test_index in splitter.split(data_x, data_y):
        # splits the data frames in test and train
        train_x, train_y = data_x.iloc[train_index], data_y.iloc[train_index]
        test_x, test_y = data_x.iloc[test_index], data_y.iloc[test_index]
        
        model.fit(train_x, train_y)

        # obtains the predictions on the test set
        predictions = model.predict(test_x)

        # calculates and reports some metrics
        acc = accuracy_score(predictions, test_y)
        prec = precision_score(predictions, test_y, average='macro')
        print('{}\t\t{:.3f}\t{:.3f}'.format(len(test_y), prec, acc))
 


data_x, data_y = generate_dataset('instances/bw_large.d.cnf')

print('Decision tree')
learner = DecisionTreeClassifier()
run_model(learner, data_x, data_y ) #default is stratified 5-fold cross validati
#run_model(learner, data_x, data_y, StratifiedShuffleSplit(n_splits=1, test_size=0.10)) # test with holdout of 10% for test